# Nature Conservancy Fish Classification - Conv Model

### Imports & environment

In [1]:
# import default libraries
import os
import glob
import shutil
import time
import argparse
import pickle

In [2]:
import numpy as np

np.random.seed(7)

In [3]:
from keras.callbacks import ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from utils import * 
from vgg16bn import Vgg16BN

Using Theano backend.
Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)
/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


### Config & Hyperparameters

In [4]:
# global parameters
args = {
    "pretrained": True,
    "datadir": "../data",
    "optim": "adam", # sgd, adam, rmsprop
    "epochs": 100,
    "batch_size": 32,
    "lr": 1e-3,
    "momentum": 0.9,
    "weight_decay": 1e-4,
    "seed": 7,
    "nb_augs": 10,
    "cv": 5
}
args = argparse.Namespace(**args)

n_filters = 158

In [5]:
# data folders
traindir_full = os.path.join(args.datadir, "train")
testdir = os.path.join(args.datadir, "test_stg1")
# intermediate folder
intermediate_path = os.path.join("..", "intermediate")
submission_path = os.path.join(intermediate_path, "submissions")
if not os.path.isdir(submission_path):
    os.makedirs(submission_path)
# get classes
classes = sorted([x.split("/")[-1] for x in glob.glob(traindir_full+"/*")])

In [6]:
testdir1 = os.path.join(intermediate_path, "test_stg1")
testdir2 = os.path.join(testdir1, "test")
if not os.path.isdir(testdir2):
    shutil.copytree(testdir, testdir2)

In [7]:
# split train/val cross validation
traindir = []
valdir = []

g = glob.glob(traindir_full + "/*/*.jpg")
gg = ["/"+x.split("/")[-2]+"/"+x.split("/")[-1] for x in g]
np.random.seed(args.seed)
shuf = np.random.permutation(gg)
ticks = []
for i in range(args.cv):
    ticks.append(i * (len(gg)//args.cv))
ticks.append(len(gg))

for i in range(args.cv):
    traindir.append(os.path.join(intermediate_path, "train{}_{}".format(
        args.cv, str(i))))
    valdir.append(os.path.join(intermediate_path, "val{}_{}".format(
        args.cv, str(i))))
    if not os.path.isdir(traindir[i]):
        shutil.copytree(traindir_full, traindir[i])
    if not os.path.isdir(valdir[i]):
        vals = shuf[ticks[i]:ticks[i+1]]
        for val in vals:
            os.renames(traindir[i] + val, valdir[i] + val)

In [8]:
def get_classes(trn_path, val_path):
    batches = get_batches(trn_path, shuffle=False, batch_size=1)
    val_batches = get_batches(val_path, shuffle=False, batch_size=1)
    return (val_batches.classes, batches.classes, onehot(val_batches.classes),
            onehot(batches.classes), val_batches.filenames, batches.filenames)

In [9]:
preds = []
conv_test_feat = load_array(intermediate_path + '/precomputed/test_ft_640.dat')
test_batches = get_batches(testdir1, shuffle=False, batch_size=1)
test_filenames = test_batches.filenames
nb_test_samples = len(test_filenames)

Found 1000 images belonging to 1 classes.


In [10]:
def get_lrg_layers():
    return [
        BatchNormalization(axis=1,
                           input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(n_filters, 3, 3, activation='relu',
                      border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(n_filters, 3, 3, activation='relu',
                      border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(n_filters, 3, 3, activation='relu',
                      border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode='same'),
        Dropout(0.5),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [11]:
def gen_preds(model):
        if args.nb_augs:
            gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.05,
                                     zoom_range=0.05, channel_shift_range=10,
                                     height_shift_range=0.05, shear_range=0.05,
                                     horizontal_flip=True)
            predictions = np.zeros(shape=(nb_test_samples, len(classes)))

            for aug in range(args.nb_augs):
                predictions += model.predict(conv_test_feat,
                                             batch_size=args.batch_size)

            predictions /= args.nb_augs
        else:
            predictions = model.predict(conv_test_feat, batch_size=batch_size)

        return predictions

In [12]:
if not os.path.isdir(os.path.join(intermediate_path, "best")):
    os.makedirs(os.path.join(intermediate_path, "best"))

In [13]:
for i in range(1, args.cv):
    (val_classes, trn_classes, val_labels, trn_labels,
     val_filenames, filenames) = get_classes(traindir[i], valdir[i])

    vgg640 = Vgg16BN((360, 640)).model
    vgg640.pop()
    vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

    train_fn = (intermediate_path
                + '/precomputed/xtrn_ft_cv{}{}_640.dat'.format(args.cv, i))
    val_fn = (intermediate_path
              + '/precomputed/xval_ft_cv{}{}_640.dat'.format(args.cv, i))
    if not (os.path.exists(train_fn) and os.path.exists(val_fn)):
        nb_split_train_samples = len(glob.glob(traindir[i] + "/*/*.jpg"))
        batches = get_batches(traindir[i], batch_size=1,
                              target_size=(360, 640), shuffle=False,
                              class_mode=None)
        conv_trn_feat = vgg640.predict_generator(batches,
                                                 nb_split_train_samples)
        save_array(train_fn, conv_trn_feat)

        nb_valid_samples = len(glob.glob(valdir[i] + "/*/*.jpg"))
        val_batches = get_batches(valdir[i], batch_size=1,
                                  target_size=(360, 640), shuffle=False,
                                  class_mode=None)
        conv_val_feat = vgg640.predict_generator(val_batches, nb_valid_samples)
        save_array(val_fn, conv_val_feat)
    else:
        conv_trn_feat = load_array(train_fn)
        conv_val_feat = load_array(val_fn)

    conv_layers, _ = split_at(vgg640, Convolution2D)
    lrg_model = Sequential(get_lrg_layers())

    lrg_model.compile(Adam(lr=args.lr), loss='categorical_crossentropy',
                      metrics=['accuracy'])

    model_fn = (intermediate_path
                + '/best/cv{}_{}_640x360_vgg16bn.h5'
                  .format(args.cv, i))
    ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)

    for j in range(4):
        nb_epoch = 2
        if j != 0:
            lrg_model.optimizer.lr /= 10
            nb_epoch = 5
        lrg_model.fit(conv_trn_feat, trn_labels, batch_size=args.batch_size,
                      nb_epoch=nb_epoch, verbose=2,
                      validation_data=(conv_val_feat, val_labels),
                      callbacks=[ckpt])

    lrg_model.load_weights(model_fn)
    preds.append(gen_preds(lrg_model))
    del lrg_model
    del conv_trn_feat
    del conv_val_feat

Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.


/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 360, 640)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.
Train on 3022 samples, validate on 755 samples
Epoch 1/2
10s - loss: 0.7406 - acc: 0.7667 - val_loss: 1.2640 - val_acc: 0.6967
Epoch 2/2
10s - loss: 0.2082 - acc: 0.9398 - val_loss: 0.2809 - val_acc: 0.9219
Train on 3022 samples, validate on 755 samples
Epoch 1/5
10s - loss: 0.0889 - acc: 0.9735 - val_loss: 0.2297 - val_acc: 0.9351
Epoch 2/5
10s - loss: 0.0753 - acc: 0.9805 - val_loss: 0.2328 - val_acc: 0.9338
Epoch 3/5
10s - loss: 0.0524 - acc: 0.9821 - val_loss: 0.2760 - val_acc: 0.9311
Epoch 4/5
10s - loss: 0.0409 - acc: 0.9894 - val_loss: 0.4914 - val_acc: 0.8556
Epoch 5/5
10s - loss: 0.0515 - acc: 0.9854 - val_loss: 0.3012 - val_acc: 0.9510
Train on 3022 samples, validate on 755 samples
Epoch 1/5
10s - loss: 0.0344 - acc: 0.9887 - val_loss: 0.2941 - val_acc: 0.9377
Epoch 2/5
10s - loss: 0.0461 - acc: 0.9884 - val_loss: 0.2563 - val_acc: 0.9603
Epoch 3/5
10s - loss: 0.0311 - acc: 0.9911 - val_loss: 0

/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 360, 640)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.
Train on 3022 samples, validate on 755 samples
Epoch 1/2
10s - loss: 0.7784 - acc: 0.7528 - val_loss: 0.6795 - val_acc: 0.8053
Epoch 2/2
10s - loss: 0.2094 - acc: 0.9351 - val_loss: 0.7747 - val_acc: 0.7669
Train on 3022 samples, validate on 755 samples
Epoch 1/5
10s - loss: 0.0906 - acc: 0.9735 - val_loss: 0.4309 - val_acc: 0.8848
Epoch 2/5
10s - loss: 0.0580 - acc: 0.9838 - val_loss: 0.3255 - val_acc: 0.9179
Epoch 3/5
10s - loss: 0.0656 - acc: 0.9798 - val_loss: 0.2891 - val_acc: 0.9404
Epoch 4/5
10s - loss: 0.0672 - acc: 0.9808 - val_loss: 0.3104 - val_acc: 0.9311
Epoch 5/5
10s - loss: 0.0441 - acc: 0.9868 - val_loss: 0.2870 - val_acc: 0.9404
Train on 3022 samples, validate on 755 samples
Epoch 1/5
10s - loss: 0.0192 - acc: 0.9960 - val_loss: 0.2116 - val_acc: 0.9510
Epoch 2/5
10s - loss: 0.0042 - acc: 0.9993 - val_loss: 0.2299 - val_acc: 0.9536
Epoch 3/5
10s - loss: 0.0035 - acc: 0.9993 - val_loss: 0

/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 360, 640)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Found 3022 images belonging to 8 classes.
Found 755 images belonging to 8 classes.
Train on 3022 samples, validate on 755 samples
Epoch 1/2
10s - loss: 0.8088 - acc: 0.7465 - val_loss: 1.3794 - val_acc: 0.6093
Epoch 2/2
10s - loss: 0.2243 - acc: 0.9381 - val_loss: 0.4533 - val_acc: 0.8781
Train on 3022 samples, validate on 755 samples
Epoch 1/5
10s - loss: 0.1085 - acc: 0.9699 - val_loss: 0.2628 - val_acc: 0.9166
Epoch 2/5
10s - loss: 0.0902 - acc: 0.9752 - val_loss: 0.3052 - val_acc: 0.9325
Epoch 3/5
10s - loss: 0.0560 - acc: 0.9848 - val_loss: 0.3489 - val_acc: 0.9192
Epoch 4/5
10s - loss: 0.0493 - acc: 0.9881 - val_loss: 0.1986 - val_acc: 0.9616
Epoch 5/5
10s - loss: 0.0232 - acc: 0.9924 - val_loss: 0.2807 - val_acc: 0.9483
Train on 3022 samples, validate on 755 samples
Epoch 1/5
10s - loss: 0.0435 - acc: 0.9901 - val_loss: 0.2733 - val_acc: 0.9298
Epoch 2/5
10s - loss: 0.0321 - acc: 0.9921 - val_loss: 0.3703 - val_acc: 0.9417
Epoch 3/5
10s - loss: 0.0378 - acc: 0.9881 - val_loss: 0

/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_4 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 360, 640)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Found 3020 images belonging to 8 classes.
Found 757 images belonging to 8 classes.
Train on 3020 samples, validate on 757 samples
Epoch 1/2
10s - loss: 0.7713 - acc: 0.7490 - val_loss: 0.6032 - val_acc: 0.7886
Epoch 2/2
10s - loss: 0.1989 - acc: 0.9454 - val_loss: 0.2104 - val_acc: 0.9564
Train on 3020 samples, validate on 757 samples
Epoch 1/5
10s - loss: 0.0929 - acc: 0.9735 - val_loss: 0.1771 - val_acc: 0.9524
Epoch 2/5
10s - loss: 0.0626 - acc: 0.9825 - val_loss: 0.2097 - val_acc: 0.9353
Epoch 3/5
10s - loss: 0.0466 - acc: 0.9861 - val_loss: 0.1777 - val_acc: 0.9577
Epoch 4/5
10s - loss: 0.0355 - acc: 0.9911 - val_loss: 0.2583 - val_acc: 0.9445
Epoch 5/5
10s - loss: 0.0256 - acc: 0.9927 - val_loss: 0.2353 - val_acc: 0.9326
Train on 3020 samples, validate on 757 samples
Epoch 1/5
10s - loss: 0.0545 - acc: 0.9871 - val_loss: 0.3236 - val_acc: 0.9036
Epoch 2/5
10s - loss: 0.0720 - acc: 0.9805 - val_loss: 0.2655 - val_acc: 0.9379
Epoch 3/5
10s - loss: 0.0485 - acc: 0.9861 - val_loss: 0

In [14]:
with open("../intermediate/best/predictions_vgg16bn.pkl", "wb") as f:
    pickle.dump(preds, f, pickle.HIGHEST_PROTOCOL)

In [17]:
def write_submission(predictions, filenames, clip=0):
    preds = [np.clip(pred, (1-clip)/7, clip) for pred in predictions]
    preds = sum(preds)/len(preds)
    sub_fn = submission_path + '/cv{}_{}augs_clip{}_vgg16bn'.format(
        args.cv, args.nb_augs, clip)

    with open(sub_fn + '.csv', 'w') as f:
        print("Writing Predictions to CSV...")
        f.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')
        for i, image_name in enumerate(filenames):
            pred = ['%.6f' % p for p in preds[i, :]]
            f.write('%s,%s\n' % (os.path.basename(image_name), ','.join(pred)))
        print("Done.")

In [18]:
write_submission(preds, test_filenames, clip=0.82)

Writing Predictions to CSV...
Done.
